install all the packages:

In [1]:
import pandas as pd
import numpy as np
from adtoolbox import core,configs,utils,adm
import plotly.express as px

In [24]:
acids=pd.read_table('../raw_data/DeLong_beto/acids_cleaned.csv',delimiter=',')
metadata=pd.read_table("../raw_data/DeLong_beto/DeLong_metadata_final[72].txt", delimiter="\t")

Filter/delete duplicates on the acid column

In [25]:
acids2adm={
    "C4":"S_bu",
    "C3":"S_pro",
    "C5":"S_va",
    "ISOC5":"S_va",
    "ISOC4":"S_bu",
    "C2":"S_ac",
    "C6":"S_cap",
    "ISOC6":"S_cap"

}

In [26]:
def identifier(items):
    if items in acids2adm:
        return acids2adm[items]

acids["admids"]=acids["acid"].apply(identifier)

In [27]:
acids2seed={
    "C4":"cpd00211",
    "C3":"cpd00141",
    "C5":"cpd00597",
    "ISOC5":"cpd00597",
    "ISOC4":"cpd00211",
    "C2":"cpd00029",
    "C6":"cpd01113",
    "ISOC6":"cpd01113"

}

In [28]:
def identity(items):
    if items in acids2seed:
        return acids2seed[items]

acids["seedids"]=acids["acid"].apply(identity)

We have to make sure we have Seed databases downloaded for COD calculations

In [29]:
from adtoolbox import core,configs
seed_db=core.SeedDB(configs.Database())
db=core.Database(configs.Database())
# db.download_seed_databases()

In [30]:
acids=acids.dropna()

In [31]:
codmap={ i:seed_db.instantiate_metabs(i).cod_calc(add_h=1) for i in acids["seedids"].unique()}

In [32]:
acids["cod_map"]=acids["seedids"].apply(lambda x:codmap[x])

In [33]:
acids['cod_conc']=acids['conc_gL']*acids['cod_map']

In [34]:
acids["shortsampleid"]=(acids.sample_id.str.split('_').apply(lambda x:"_".join(x[:5])))


We don't want to simulate for Control and inoculum only samples

In [36]:
acids

,sample_id,acid,conc_gL,day,T,pH,admids,seedids,cod_map,cod_conc,shortsampleid
0,A_35_F_5_A_D0,C2,0.106018,0.0,35.0,5.0,S_ac,cpd00029,1.066667,0.113086,A_35_F_5_A
1,A_35_F_5_A_D1,C2,0.052312,1.2,35.0,5.0,S_ac,cpd00029,1.066667,0.055800,A_35_F_5_A
2,A_35_F_5_A_D1,C4,0.988501,1.2,35.0,5.0,S_bu,cpd00211,1.818182,1.797275,A_35_F_5_A
3,A_35_F_5_A_D1,C6,0.129533,1.2,35.0,5.0,S_cap,cpd01113,2.206897,0.285866,A_35_F_5_A
4,A_35_F_5_A_D2,C2,0.868033,3.3,35.0,5.0,S_ac,cpd00029,1.066667,0.925902,A_35_F_5_A
...,...,...,...,...,...,...,...,...,...,...,...
1967,T_55_M_9_C_D4,C3,0.000000,10.5,55.0,9.0,S_pro,cpd00141,1.513514,0.000000,T_55_M_9_C
1968,T_55_M_9_C_D4,C4,0.000000,10.5,55.0,9.0,S_bu,cpd00211,1.818182,0.000000,T_55_M_9_C
1970,T_55_M_9_C_D5,C2,0.815572,14.5,55.0,9.0,S_ac,cpd00029,1.066667,0.869943,T_55_M_9_C
1971,T_55_M_9_C_D5,C3,0.000000,14.5,55.0,9.0,S_pro,cpd00141,1.513514,0.000000,T_55_M_9_C


In [100]:
pivot_df = acids.pivot_table(index=["shortsampleid", "day"], columns="acid", values="cod_conc", fill_value=0)
pivot_df["C4"]=pivot_df["C4"]+pivot_df["ISOC4"]
pivot_df["C5"]=pivot_df["C5"]+pivot_df["ISOC5"]
pivot_df["C6"]=pivot_df["C6"]+pivot_df["ISOC6"]
pivot_df=pivot_df.drop(columns=["ISOC4","ISOC5","ISOC6"])
pivot_df.rename(columns={"C4":"S_bu","C3":"S_pro","C5":"S_va","C2":"S_ac","C6":"S_cap"},inplace=True)
data_dict = {}
for reactor in pivot_df.index.levels[0]:
  data_dict[reactor] = {}
  data_dict[reactor]["time"]=pivot_df.loc[reactor].index.tolist()
  data_dict[reactor]["data"]=pivot_df.loc[reactor].to_dict(orient="list")


In [101]:
cod_counts=pd.read_csv('./COD_counts.csv')

In [102]:
data_i={}
for i in cod_counts.columns[cod_counts.columns.str.endswith("_D0")]:
  data_i[i]=cod_counts.set_index("Unnamed: 0")[i].to_dict()



In [104]:
num_failed=0
pivot_df = acids.pivot_table(index=["shortsampleid", "day"], columns="acid", values="cod_conc", fill_value=0)
pivot_df["C4"]=pivot_df["C4"]+pivot_df["ISOC4"]
pivot_df["C5"]=pivot_df["C5"]+pivot_df["ISOC5"]
pivot_df["C6"]=pivot_df["C6"]+pivot_df["ISOC6"]
pivot_df=pivot_df.drop(columns=["ISOC4","ISOC5","ISOC6"])
pivot_df.rename(columns={"C4":"S_bu","C3":"S_pro","C5":"S_va","C2":"S_ac","C6":"S_cap"},inplace=True)
data_dict = {}
for reactor in pivot_df.index.levels[0]:
  data_dict[reactor] = {}
  data_dict[reactor]["time"]=pivot_df.loc[reactor].index.tolist()
  data_dict[reactor]["data"]=pivot_df.loc[reactor].to_dict(orient="list")

  try:
    data_dict[reactor]["Initial conditions"]=cod_counts.set_index("Unnamed: 0")[reactor+"_D0"].to_dict()
  except:
    num_failed+=1
    print("Failure: ",num_failed,":",reactor)
      

Failure:  1 : A_35_F_5_B
Failure:  2 : A_35_F_5_D
Failure:  3 : A_35_F_7_D
Failure:  4 : A_35_F_9_D
Failure:  5 : A_35_M_5_A
Failure:  6 : A_35_M_5_D
Failure:  7 : A_35_M_7_B
Failure:  8 : A_35_M_7_C
Failure:  9 : A_35_M_7_D
Failure:  10 : A_35_M_9_D
Failure:  11 : A_45_F_5_A
Failure:  12 : A_45_F_5_B
Failure:  13 : A_45_F_5_C
Failure:  14 : A_45_F_5_D
Failure:  15 : A_45_F_7_D
Failure:  16 : A_45_F_9_D
Failure:  17 : A_45_M_5_A
Failure:  18 : A_45_M_5_B
Failure:  19 : A_45_M_5_D
Failure:  20 : A_45_M_7_D
Failure:  21 : A_45_M_9_A
Failure:  22 : A_45_M_9_B
Failure:  23 : A_45_M_9_C
Failure:  24 : A_45_M_9_D
Failure:  25 : B_35_F_5_D
Failure:  26 : B_35_F_9_D
Failure:  27 : B_35_M_5_D
Failure:  28 : B_35_M_7_D
Failure:  29 : B_35_M_9_B
Failure:  30 : B_35_M_9_D
Failure:  31 : B_45_F_5_B
Failure:  32 : B_45_F_5_C
Failure:  33 : B_45_F_5_D
Failure:  34 : B_45_F_7_A
Failure:  35 : B_45_F_7_C
Failure:  36 : B_45_F_9_A
Failure:  37 : B_45_F_9_C
Failure:  38 : B_45_F_9_D
Failure:  39 : B_45_M

In [105]:
feed_manure = metadata[metadata['feedstock']==('M')]

In [106]:
feed_manure=feed_manure['feedstock']

In [107]:
feed_waste = metadata[metadata['feedstock']==('F')]

In [108]:
feed_waste=feed_waste['feedstock']

In [109]:
manure = core.Feed(
        name="Manure",
        lipids=10.7,
        proteins=17.3,
        carbohydrates=70,
        xi= 60,
        si=50,
        tss=80,
)

In [110]:
foodwaste = core.Feed(
        name="foodwaste",
        lipids=21,
        proteins=20.5,
        carbohydrates=50.1,
        xi=60,
        si=50,
        tss=65
)

In [96]:
core.Database(configs.Database(
    adm_parameters_urls=configs.E_ADM_2_REMOTE,
    adm_parameters=configs.E_ADM_2_LOCAL
)).download_adm_parameters()


model_parameters downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramet
ers/e_adm_2_model_parameters.json

base_parameters downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramete
rs/e_adm_2_base_parameters.json

initial_conditions downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Param
eters/e_adm_2_initial_conditions.json

inlet_conditions downloaded to /Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Paramet
ers/e_adm_2_inlet_conditions.json

reactions downloaded to 
/Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Parameters/e_adm_2_reactions.json

species downloaded to 
/Users/parsaghadermarzi/Desktop/ADToolbox/Database/ADM_Parameters/e_adm_2_species.json

In [111]:
params=utils.load_multiple_json_files(configs.E_ADM_2_LOCAL)

We need a definition of the feedstock

In [128]:
Manure = core.Feed(
        name="Manure",
        lipids=10.7,
        proteins=30,
        carbohydrates=59.3,
        xi= 80,
        si=50,
        tss=95,
 )
foodwaste = core.Feed(
        name="foodwaste",
        lipids=21,
        proteins=20.5,
        carbohydrates=50.1,
        xi=60,
        si=50,
        tss=65
)

In [129]:
results_dict = {}
# dash_apps = {}
inoculum_cod=10
total_feed_cod=15
params.base_parameters['q_in'] = 0
mgo=core.Database(config=configs.Database())
for reactor_name, reactor_data in data_dict.items():
    if "Initial conditions" in reactor_data:
        initial_conditions = {key:value*inoculum_cod for key,value in reactor_data["Initial conditions"].items()}
        initial_conditions["X_VFA_deg"]=0.1*inoculum_cod
        _, t, f, ph, _ = reactor_name.split("_")
        print(float(t), f, float(ph),end="\r")
        if f == "F":
            feed = foodwaste
        else:
            feed = Manure
        initial_conditions["TSS"]=total_feed_cod*feed.tss/100
        initial_conditions["TDS"]=total_feed_cod-initial_conditions["TSS"]
        initial_conditions["s_su"]=0
        initial_conditions["s_aa"]=0
        initial_conditions["s_va"]=0
        initial_conditions["s_fa"]=0
        initial_conditions["s_ac"]=0
        initial_conditions["s_cap"]=0
        if reactor_data["time"][0] != 0:
            reactor_data["time"].insert(0,0)
            for key in reactor_data["data"]:
                reactor_data["data"][key].insert(0,0)
        
        for key,value in reactor_data["data"].items():
            initial_conditions[key]=value[0]
        

        params.initial_conditions.update(initial_conditions)
        
        
        temp_exp_obj=core.Experiment(
            name=reactor_name+"_Delong_BETO",
            time=reactor_data["time"],
            variables=list(reactor_data["data"].keys()),
            data=list(reactor_data["data"].values()),
            initial_concentrations=params.initial_conditions,
            reference="Delong_Lab_BETO_Experiment",
            model_name="e_adm_2"
        )
        
        mgo.add_experiment_to_experiments_db(temp_exp_obj)

        model = adm.Model(
            model_parameters=params.model_parameters,
            base_parameters=params.base_parameters,
            initial_conditions=params.initial_conditions,
            inlet_conditions=params.inlet_conditions,
            species=params.species,
            reactions=params.reactions,
            feed=feed,
            ode_system=adm.e_adm_2_ode_sys,
            build_stoichiometric_matrix=adm.build_e_adm_2_stoichiometric_matrix,
            time_limit=100000,
            control_state={'S_H_ion': 10 ** -(float(ph))})

        sol = model.solve_model(np.linspace(0, 20, 10))
        results_dict[reactor_name] = pd.DataFrame(sol.y.T,columns=params.species)



In [130]:
only_initial_conditions = {}
for data, initial in data_dict.items():
    if "Initial conditions" in initial:
        only_initial_conditions[data] = pd.DataFrame(initial["data"]).max()

In [131]:
exp_data_sim=pd.DataFrame(only_initial_conditions).T

In [132]:
exp_data_sim["Feed"]=exp_data_sim.index.map(lambda x:x.split("_")[2])

In [133]:
exp_data_sim["PH"]=exp_data_sim.index.map(lambda x:x.split("_")[3])

In [134]:
fig=px.box(exp_data_sim,x="Feed",y="S_bu",facet_col="PH",color="Feed",points="all")
fig.update_layout(title="Butyric Acid Concentration Experimental Data")
fig.write_html("Butyric Acid Concentratio Experimental Data.html")
fig.show()

In [135]:
observed_acids=exp_data_sim.columns[:-2].to_list()
for reactor in results_dict:
  results_dict[reactor]=results_dict[reactor][observed_acids].max()


In [136]:
simulations_vfa_results=pd.DataFrame(results_dict).T

In [137]:
simulations_vfa_results["Feed"]=simulations_vfa_results.index.map(lambda x:x.split("_")[2])

In [138]:
simulations_vfa_results["PH"]=simulations_vfa_results.index.map(lambda x:x.split("_")[3])

In [139]:
simulations_vfa_results

,S_ac,S_pro,S_bu,S_va,S_cap,Feed,PH
A_35_F_5_A,1.346559,2.772614,1.164341,2.448643,0.192994,F,5
A_35_F_5_C,1.361359,2.777306,1.177625,2.458072,0.193762,F,5
A_35_F_7_A,1.450354,2.756807,1.086939,2.490990,0.184214,F,7
A_35_F_7_B,1.533122,2.752617,1.163869,2.618100,0.193759,F,7
A_35_F_7_C,1.529489,2.745193,1.165847,2.658041,0.196468,F,7
...,...,...,...,...,...,...,...
T_55_M_7_A,0.894132,1.415668,0.617391,1.438023,0.103538,M,7
T_55_M_7_B,0.815144,1.394894,0.567658,1.445022,0.103697,M,7
T_55_M_7_C,0.761750,1.394867,0.535179,1.465816,0.105294,M,7
T_55_M_9_B,1.577458,0.878959,0.205256,2.027516,0.128091,M,9


In [140]:
fig=px.box(simulations_vfa_results,x="Feed",y="S_bu",facet_col="PH",color="Feed",points="all")
fig.update_layout(title="Simulated Butyric Acid concentrations")
fig.write_html("simulated_vfa_concentrations.html")
fig.show()

6